In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")

/home/yuhao/anaconda3/envs/easycontext/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/yuhao/anaconda3/envs/easycontext/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `PYTORCH_TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.68it/s]


In [3]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU instead.")

# 将模型移至设定的设备（GPU或CPU）
model = model.to(device)

# 设置输入指令
input_text = "Olivia is a photographer. He likes to keep a weekly journal, documenting:\n1) Family life with: wife (birthday on May 04), child_1 (birthday on December 20), child_2 (birthday on January 16), child_3 (birthday on August 21).\n2) Attending a week-long jazz festival in Montreal in week 32.\n3) Join a public speaking and debate club every 3 weeks on weekends. No less than 100 word weekly diary. \nPlease start generating his weekly diary from January until the end of the year without interruption."


# 将模型和数据移至设定的设备
model = model.to(device)
input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

# 现在模型运行在GPU上，可以继续进行生成等操作
output_ids = model.generate(input_ids, max_length=16000)

# 注意：生成完成后如果需要处理或打印输出，可能需要将数据移回CPU
output_ids = output_ids.to('cpu')
decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(decoded_output)



Using GPU: NVIDIA A100 80GB PCIe


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Olivia is a photographer. He likes to keep a weekly journal, documenting:
1) Family life with: wife (birthday on May 04), child_1 (birthday on December 20), child_2 (birthday on January 16), child_3 (birthday on August 21).
2) Attending a week-long jazz festival in Montreal in week 32.
3) Join a public speaking and debate club every 3 weeks on weekends. No less than 100 word weekly diary. 
Please start generating his weekly diary from January until the end of the year without interruption. 

Week 1:
I went to the jazz festival in Montreal last week. Today, I attended the public speaking and debate club. I gave a short speech about photography. The club members were very supportive. At home, I tried to capture some family moments, especially with child_1 who is very excited about the new year. I took some photos of them playing together. My wife is looking forward to celebrating her birthday on May 04.

Week 2:
Today was a busy day. I attended the public speaking and debate club again. 

In [1]:
from vllm import LLM, SamplingParams
import torch
# Sample prompts.
prompts = [
    "Olivia is a photographer. He likes to keep a weekly journal, documenting:\n1) Family life with: wife (birthday on May 04), child_1 (birthday on December 20), child_2 (birthday on January 16), child_3 (birthday on August 21).\n2) Attending a week-long jazz festival in Montreal in week 32.\n3) Join a public speaking and debate club every 3 weeks on weekends. No less than 100 word weekly diary. \nPlease start generating his weekly diary from January until the end of the year(2012) without interruption."
]
# Create a sampling params object.
device = "cuda" if torch.cuda.is_available() else "cpu"
sampling_params = SamplingParams(temperature=0.8, top_p=0.95,max_tokens = 5000)

# Create an LLM.
llm = LLM(model="meta-llama/Meta-Llama-3.1-8B-Instruct",device=device)
# Generate texts from the prompts. The output is a list of RequestOutput objects
# that contain the prompt, generated text, and other information.
outputs = llm.generate(prompts, sampling_params)
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

/home/yuhao/anaconda3/envs/easycontext/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/yuhao/anaconda3/envs/easycontext/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `PYTORCH_TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2024-07-29 15:46:18,081	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


WARNING 07-29 15:46:19 arg_utils.py:762] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 07-29 15:46:19 config.py:806] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 07-29 15:46:19 llm_engine.py:176] Initializing an LLM engine (v0.5.3.post1) with config: model='meta-llama/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_p

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:01,  1.59it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.28it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  1.76it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.59it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.56it/s]



INFO 07-29 15:46:24 model_runner.py:692] Loading model weights took 14.9888 GB
INFO 07-29 15:46:25 gpu_executor.py:102] # GPU blocks: 28238, # CPU blocks: 2048
INFO 07-29 15:46:26 model_runner.py:980] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 07-29 15:46:26 model_runner.py:984] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 07-29 15:46:32 model_runner.py:1181] Graph capturing finished in 5 secs.


Processed prompts: 100%|██████████| 1/1 [01:06<00:00, 66.89s/it, est. speed input: 1.85 toks/s, output: 74.75 toks/s]

Prompt: 'Olivia is a photographer. He likes to keep a weekly journal, documenting:\n1) Family life with: wife (birthday on May 04), child_1 (birthday on December 20), child_2 (birthday on January 16), child_3 (birthday on August 21).\n2) Attending a week-long jazz festival in Montreal in week 32.\n3) Join a public speaking and debate club every 3 weeks on weekends. No less than 100 word weekly diary. \nPlease start generating his weekly diary from January until the end of the year(2012) without interruption.', Generated text: ' \nJournal entry for January 01: Happy New Year! Spent the day with my family and enjoyed the long weekend. \nJournal entry for January 08: The debate club met today. We did a great exercise on persuading the audience. The child_1 was quite interested and asked lots of questions.\nJournal entry for January 15: Child_2\'s birthday was today. We had a great celebration. Wish my wife was here, she was out for work.\nJournal entry for January 22: Did some outdoor pho

: 

In [2]:
import os
from together import Together
export TOGETHER_API_KEY='41be8838bb6c8b2329651388695f038a6ec87ab2bb13819af98dd67cf36c86d5'
client = Together(api_key=os.environ.get("TOGETHER_API_KEY"))

stream = client.chat.completions.create(
  model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
  messages=[{"role": "user", "content": "What are some fun things to do in New York?"}],
  stream=True,
)

for chunk in stream:
  print(chunk.choices[0].delta.content or "", end="", flush=True)

SyntaxError: invalid syntax (65786602.py, line 3)

In [1]:
import os
from together import Together

client = Together(api_key=os.environ.get("TOGETHER_API_KEY"))

stream = client.chat.completions.create(
  model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
  messages=[{"role": "user", "content": "What are some fun things to do in New York?"}],
  stream=True,
)

for chunk in stream:
  print(chunk.choices[0].delta.content or "", end="", flush=True)

AuthenticationError: The api_key client option must be set either by passing api_key to the client or by setting the TOGETHER_API_KEY environment variable